# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [9]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [16]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

personal preference and varies from person to person. Some people prefer dogs because they are

--------------------------------------------------------------------------------

Max Tokens: 32

matter of personal preference and lifestyle. Each pet has its own unique characteristics and requirements, so it's important to consider what kind of pet will fit best with your

--------------------------------------------------------------------------------

Max Tokens: 60

subjective opinion and can vary depending on the person's lifestyle, preferences, and needs. Some people might prefer dogs for their loyalty and protection, while others might prefer cats for their independence and low-maintenance. Some people might prefer small animals like hamsters or guinea pigs for their adorable and cudd

--------------------------------------------------------------------------------

Max Tokens: 100

subjective matter that varies from person to person. Different people have different 

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [10]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [11]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

subjective topic as it depends on personal preference and lifestyle. Some people may prefer dogs as they provide companionship and can be trained for various tasks. Others may prefer cats as they are more independent and require less maintenance. Some people may prefer smaller pets like hamsters or fish as they are easier to
subjective matter and varies from person to person based on their preferences, lifestyle, and living conditions. Some people might prefer dogs or cats, while others might prefer birds, fish, or small mammals like hamsters or guinea pigs. It's important to do research and consider all factors before choosing a
That depends on your lifestyle, living situation, and personal preferences. Some people may prefer dogs because they are loyal and active, while others may prefer cats because they are independent and low-maintenance. Other popular pets include birds, fish, rabbits, and rodents like hamsters and guinea pigs. It
matter of personal preference and lifestyle. Some

In [12]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

subjective matter and depends on personal preferences and lifestyle. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low-maintenance nature. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet is one that fits
subjective matter and depends on personal preferences and lifestyle. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low-maintenance nature. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet is one that fits
subjective matter and depends on personal preferences and lifestyle. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low-maintenance nature. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet is one that fits
subjective matter and depends on personal p

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [13]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "The best pet is a "}],
                max_tokens=60,
                n=2
        )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 personal preference, as different people have different needs and preferences for pets. Some popular pets include dogs, cats, fish, birds, hamsters, and rabbits. It's important to consider factors such as time, space, budget, and lifestyle when choosing a pet. Ultimately, the best pet is one
1 companion that suits your lifestyle and preferences. Some popular choices include cats, dogs, rabbits, birds, fish, and even reptiles. It's essential to consider the time, effort, and resources required to care for a pet before making a decision. You should also consider factors such as your living


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [15]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty=presence_penalty
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

matter of personal preference, some people prefer cats, while others prefer dogs or other types of animals. It's essential to consider factors such as your lifestyle, living situation, and the amount of time you can dedicate to taking care of a pet before adopting any animal. It's also important to research the

--------------------------------------------------------------------------------

Presence Penalty: 0.5

matter of personal preference and lifestyle. Some people might prefer a dog for their loyalty and companionship, while others might prefer a cat for their independence and low-maintenance care. Other popular pets include birds, fish, rabbits, and guinea pigs. It ultimately depends on what kind of pet fits with

--------------------------------------------------------------------------------

Presence Penalty: 1.0

subjective decision that depends on individual preferences and lifestyles. Some people may prefer dogs as pets for their loyalty and companion

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.